# LangGraph Github Repository 기반 Q&A 시스템 구축

- Github Repository: https://github.com/langchain-ai/langgraph

1. "Code" - "Download ZIP" 버튼을 눌러 다운로드 받습니다.
2. 다운로드 받은 파일을 압축 해제합니다.

혹은 아래 명령어로 repository 를 다운로드 받습니다.

```bash
git clone https://github.com/langchain-ai/langgraph.git
```

## 환경 설정

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

In [ ]:
from langchain_teddynote.models import get_model_name, LLMs

# 최신 LLM 모델 이름 가져오기
MODEL_NAME = get_model_name(LLMs.GPT4o)
print(MODEL_NAME)

## 데이터 전처리

먼저, langgraph 의 다운로드 받은 github repository 의 파일을 로드합니다.

여기서 로드하는 파일은 파이썬 파일(.py), 마크다운 파일(.md), 노트북 파일(.ipynb) 입니다.

In [4]:
import os
import glob
from langchain_text_splitters import Language
from langchain_community.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

import utils
import os

# 프로젝트 루트 경로
ROOT_PATH = "/Users/teddy/Dev/github/langgraph-main"
# ROOT_PATH = r"C:\Users\teddy/Dev/github/langgraph-main"

# 가져올 문서 경로
libs_path = ROOT_PATH + "/libs"
docs_path = ROOT_PATH + "/docs"
examples_path = ROOT_PATH + "/examples"

all_repos = [
    libs_path,
    docs_path,
    examples_path,
]

이제 각 형식의 문서를 로드합니다.

In [ ]:
# 파이썬 파일 로드
def load_python_files(repos):
    documents = []
    for repo in repos:
        loader = GenericLoader.from_filesystem(
            repo,
            glob="**/*",
            suffixes=[".py"],
            parser=LanguageParser(language=Language.PYTHON, parser_threshold=30),
        )
        documents.extend(loader.load())
    print(f".py 파일의 개수: {len(documents)}")
    return documents


# 마크다운 파일 로드
def load_markdown_files(repos):
    documents = []
    for repo in repos:
        try:
            loader = DirectoryLoader(
                repo,
                glob="**/*.md",
                loader_cls=TextLoader,
                loader_kwargs={"encoding": "utf-8"},
                recursive=True,
            )
            documents.extend(loader.load())
        except Exception as e:
            print(f"Error loading files from {repo}: {str(e)}")
            continue
    print(f".md 파일의 개수: {len(documents)}")
    return documents


# 노트북 파일 로드
def load_notebook_files(root_path):
    notebook_files = glob.glob(root_path + "/**/*.ipynb", recursive=True)

    # 노트북 파일을 마크다운 파일로 변환
    notebook_md_files = []
    for f in notebook_files:
        converted_file = utils.convert_notebook_to_md(f)
        notebook_md_files.append(converted_file)
    notebook_md_files = list(set(notebook_md_files))

    # 마크다운 파일 로드
    documents = []
    for f in notebook_md_files:
        loader = TextLoader(f, encoding="utf-8")
        docs = loader.load()
        for doc in docs:
            doc.metadata["source"] = doc.metadata["source"].replace(".md", ".ipynb")
        documents.extend(docs)
        os.remove(f)

    print(f".ipynb 파일의 개수: {len(documents)}")
    return documents


# 모든 문서 로드
py_documents = load_python_files(all_repos)
md_documents = load_markdown_files(all_repos)
notebook_documents = load_notebook_files(ROOT_PATH)

전체 문서의 개수를 출력합니다.

In [ ]:
all_docs = py_documents + md_documents + notebook_documents

len(all_docs)

### 문서 분할

Python 코드 파일은 문서 분할을 위해 RecursiveCharacterTextSplitter 를 사용합니다. 여기서 `Language.PYTHON` 은 파이썬 코드를 분할하는 데 사용됩니다.

Text 형식의 파일은 RecursiveCharacterTextSplitter 에 별도의 옵션을 지정하지 않습니다.

In [7]:
# 문서 분할을 위한 텍스트 스플리터 임포트
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter

# Python 코드 전용 스플리터 생성
# chunk_size: 각 청크의 최대 크기(문자 수)
# chunk_overlap: 청크 간 중복되는 문자 수
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)

# 일반 텍스트용 스플리터 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# 각 문서 타입별로 분할 수행
split_py_documents = python_splitter.split_documents(py_documents)  # Python 파일 분할
split_md_documents = text_splitter.split_documents(md_documents)  # Markdown 파일 분할
split_notebook_documents = text_splitter.split_documents(
    notebook_documents
)  # Notebook 파일 분할

In [ ]:
# 모든 문서를 합칩니다.
split_docs = split_md_documents + split_md_documents + split_notebook_documents
len(split_docs)

### 임베딩 & 벡터 DB 에 저장

다음으로는 분할한 문서를 임베딩하여 벡터 DB 에 저장합니다.

In [9]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(split_docs, embeddings)

DB index 이름을 `LANGCHAIN_DB_INDEX` 로 설정하고 저장합니다.

추후에 이 index 를 사용하여 저장된 벡터 DB 를 로드합니다.

In [10]:
# 벡터 DB 저장
DB_INDEX = "LANGCHAIN_DB_INDEX"
db.save_local(DB_INDEX)

아래는 저장된 DB 를 로드하는 예시입니다.

In [11]:
# 저장된 DB 로드
langgraph_db = FAISS.load_local(
    DB_INDEX, embeddings, allow_dangerous_deserialization=True
)

잘 저장이 되었는지 테스트를 위하여 Query 를 입력하여 결과를 확인 합니다.

In [ ]:
print(langgraph_db.similarity_search("self-rag")[0].page_content)

## RAG 체인 구축

여기서는 Naive 형태의 Chain 을 구성하도록 하겠습니다.

단, Reranker 를 사용하여 검색 정확도를 향상시키도록 하겠습니다. (`Jina-reranker` 를 사용하지만, 다른 reranker 를 사용하거나, 빼도 좋습니다.)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from operator import itemgetter

# retriever 생성
code_retriever = langgraph_db.as_retriever(search_kwargs={"k": 20})

# JinaRerank 설정
compressor = JinaRerank(model="jina-reranker-v2-base-multilingual", top_n=8)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=code_retriever
)

# prompt 설정
system_prompt = """You are an CODE Copilot Assistant. You must use the following pieces of retrieved source code or documentation to answer the question. 
You are given question related to RAG(Retrieval Augmented Generation) source code and documentation.
If you don't know the answer, just say that you don't know. Answer in Korean.

When answering questions, follow these guidelines:

1. Use only the information provided in the context. 
2. Include as many example code snippets as possible.
3. Writing a full code snippet is highly recommended.
4. Do not introduce external information or make assumptions beyond what is explicitly stated in the context.
5. The context contain sources at the topic of each individual document.
6. Include these sources your answer next to any relevant statements. For example, for source # 1 use [1]. 
7. List your sources in order at the bottom of your answer. [1] Source 1, [2] Source 2, etc
8. If the source is: <source>assistant/docs/llama3_1.md" page="7"</source>' then just list: 
        
[1] llama3_1.md
        
And skip the addition of the brackets as well as the Document source preamble in your citation.

----

### Sources

In the Sources section:
- Include all sources used in your answer
- Provide full links to relevant websites or document names
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

**Sources**
- [1] Link or Document name
- [2] Link or Document name

Be sure to combine sources. For example this is not correct:

- [3] https://ai.meta.com/blog/meta-llama-3-1/
- [4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

- [3] https://ai.meta.com/blog/meta-llama-3-1/

-----

### Retrieved Context

Here is the context that you can use to answer the question:

{context}

----

### Question

Here is user's question:

{question}

----

Final review:
- Ensure the report follows the required structure
- Check that all guidelines have been followed
- Check if a full code snippet is included in your answer if applicable.
- Your response should be written in Korean
- Using many example code snippets would be rewarded by the user
- Think step by step.

----

Your answer to the question with the source:"""

rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        ("human", "{question}"),
    ]
)

# llm 설정
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)

# 단계 8: 체인(Chain) 생성
rag_chain = (
    {
        "question": itemgetter("question"),
        "context": itemgetter("context"),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

## LangGraph 체인 구축을 위한 노드 정의

### Routing

사용자의 질문에 대한 routing 을 수행합니다.

질문이 vector 검색이 필요한지 여부를 판단하여 라우팅하며, 최대한 보수적으로 판단하도록 프롬프트를 설정하였습니다.

In [14]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class RouteQuery(BaseModel):

    # 데이터 소스 선택을 위한 리터럴 타입 필드
    binary_score: Literal["yes", "no"] = Field(
        ...,
        description="Given a user question, determine if it needs to be retrieved from vectorstore or not. Return 'yes' if it needs to be retrieved from vectorstore, otherwise return 'no'.",
    )


# LLM 초기화 및 함수 호출을 통한 구조화된 출력 생성
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# 시스템 메시지와 사용자 질문을 포함한 프롬프트 템플릿 생성
system = """You are an expert at routing a user question. 
The vectorstore contains documents related to RAG(Retrieval Augmented Generation) source code and documentation.
Return 'yes' if the question is related to the source code or documentation, otherwise return 'no'.
If you can't determine if the question is related to the source code or documentation, return 'yes'.
If you don't know the answer, return 'yes'."""

# Routing 을 위한 프롬프트 템플릿 생성
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# 프롬프트 템플릿과 구조화된 LLM 라우터를 결합하여 질문 라우터 생성
question_router = route_prompt | structured_llm_router

몇 개의 Query 를 입력하여 결과를 확인 합니다.

In [ ]:
question_router.invoke({"question": "Self-RAG 에 대해 설명하세요."})

In [ ]:
question_router.invoke({"question": "MemorySaver 에 대해 설명하세요."})

In [ ]:
question_router.invoke({"question": "대한민국의 수도는?"})

### Query Rewrite

다음으로는 사용자의 질문에 대한 질문을 재작성(query rewrite)하는 노드를 구성합니다.

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

# Query Rewrite 시스템 프롬프트
system = """You a question re-writer that converts an input question to a better version that is optimized for CODE SEARCH(github repository). 
Look at the input and try to reason about the underlying semantic intent / meaning.

Base Code Repository: 

https://github.com/langchain-ai/langgraph

Output should be in English."""

# 프롬프트 정의
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

# Question Re-writer 체인 초기화
question_rewriter = re_write_prompt | llm | StrOutputParser()

결과를 확인 합니다.

In [ ]:
question_rewriter.invoke({"question": "Self-RAG 에 대해 설명하세요."})

## 평가

중간 단계에 문서의 관련성 평가와 답변의 환각 여부 평가를 수행합니다. 따라서 다음의 3개 노드가 추가로 필요합니다.

- 검색된 문서의 관련성 평가
- 답변의 환각 여부 평가
- 답변의 질문에 대한 관련성 평가

### 검색된 문서의 평가

In [20]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


# 문서 평가를 위한 데이터 모델 정의
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM 초기화 및 함수 호출을 통한 구조화된 출력 생성
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 시스템 메시지와 사용자 질문을 포함한 프롬프트 템플릿 생성
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# 문서 검색결과 평가기 생성
retrieval_grader = grade_prompt | structured_llm_grader

### 답변의 환각 여부 평가

In [21]:
# 할루시네이션 체크를 위한 데이터 모델 정의
class AnswerGroundedness(BaseModel):
    """Binary score for answer groundedness."""

    binary_score: str = Field(
        description="Answer is grounded in the facts(given context), 'yes' or 'no'"
    )


# LLM 설정
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
structured_llm_grader = llm.with_structured_output(AnswerGroundedness)

# 프롬프트 설정
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

# 프롬프트 템플릿 생성
groundedness_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

# 답변의 환각 여부 평가기 생성
groundedness_checker = groundedness_prompt | structured_llm_grader

### 답변의 질문에 대한 관련성 평가

In [22]:
class GradeAnswer(BaseModel):
    """Binary scoring to evaluate the appropriateness of answers to questions"""

    binary_score: str = Field(
        description="Indicate 'yes' or 'no' whether the answer solves the question"
    )


# 함수 호출을 통한 LLM 초기화
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# 프롬프트 설정
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
relevant_answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

# 프롬프트 템플릿과 구조화된 LLM 평가기를 결합하여 답변 평가기 생성
relevant_answer_checker = relevant_answer_prompt | structured_llm_grader

## 도구

여기서는 TavilySearch 를 사용하여 웹 검색을 수행합니다.

In [23]:
from langchain_teddynote.tools.tavily import TavilySearch

# 웹 검색 도구 생성
web_search_tool = TavilySearch(max_results=6)

## 상태 정의

In [24]:
from typing import List
from typing_extensions import TypedDict, Annotated


# 그래프의 상태 정의
class GraphState(TypedDict):
    """
    그래프의 상태를 나타내는 데이터 모델

    Attributes:
        question: 질문
        generation: LLM 생성된 답변
        documents: 도큐먼트 리스트
    """

    question: Annotated[str, "User question"]
    generation: Annotated[str, "LLM generated answer"]
    documents: Annotated[List[str], "List of documents"]

## 노드 정의

In [25]:
from langchain_core.documents import Document


# 질문 라우팅 노드
def route_question_node(state):
    question = state["question"]
    evaluation = question_router.invoke({"question": question})

    # 질문 라우팅 결과에 따른 노드 라우팅
    if evaluation.binary_score == "yes":
        return "query_expansion"
    else:
        return "general_answer"


# 질문 재작성 노드
def query_rewrite_node(state):
    # 질문과 문서 검색 결과 가져오기
    question = state["question"]

    # 질문 재작성
    better_question = question_rewriter.invoke({"question": question})
    return {"question": better_question}


# 문서 검색 노드
def retrieve_node(state):
    question = state["question"]

    # 문서 검색 수행
    documents = compression_retriever.invoke(question)
    return {"documents": documents}


# 답변 생성 노드
def general_answer_node(state):
    # 질문 가져오기
    question = state["question"]

    # RAG 답변 생성
    answer = llm.invoke(question)
    return {"generation": answer}


# RAG 답변 생성 노드
def rag_answer_node(state):
    # 질문과 문서 검색 결과 가져오기
    question = state["question"]
    documents = state["documents"]

    # RAG 답변 생성
    answer = rag_chain.invoke({"context": documents, "question": question})
    return {"generation": answer}


# 문서의 관련성을 평가 후 필터링
def filtering_documents_node(state):
    # 질문과 문서 검색 결과 가져오기
    question = state["question"]
    documents = state["documents"]

    # 각 문서에 대한 관련성 점수 계산
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            # 관련성이 있는 문서 추가
            filtered_docs.append(d)
        else:
            # 관련성이 없는 문서는 건너뛰기
            continue

    return {"documents": filtered_docs}


# 웹 검색 노드
def web_search_node(state):
    # 질문과 문서 검색 결과 가져오기
    question = state["question"]

    # 웹 검색 수행
    web_results = web_search_tool.invoke({"query": question})
    web_results_docs = [
        Document(
            page_content=web_result["content"],
            metadata={"source": web_result["url"]},
        )
        for web_result in web_results
    ]

    return {"documents": web_results_docs}


# 추가 정보 검색 필요성 여부 평가 노드
def decide_to_web_search_node(state):
    # 문서 검색 결과 가져오기
    filtered_docs = state["documents"]

    if len(filtered_docs) < 2:
        return "web_search"
    else:
        return "rag_answer"


# 답변의 환각 여부/관련성 여부 평가 노드
def answer_groundedness_check(state):
    # 질문과 문서 검색 결과 가져오기
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # Groundedness 평가
    score = groundedness_checker.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Groundedness 평가 결과에 따른 처리
    if grade == "yes":
        # 답변의 관련성(Relevance) 평가
        score = relevant_answer_checker.invoke(
            {"question": question, "generation": generation}
        )
        grade = score.binary_score

        # 관련성 평가 결과에 따른 처리
        if grade == "yes":
            return "relevant"
        else:
            return "not relevant"

    else:
        return "not grounded"

In [26]:
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

# 그래프 상태 초기화
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("query_expand", query_rewrite_node)  # 질문 재작성
workflow.add_node("query_rewrite", query_rewrite_node)  # 질문 재작성
workflow.add_node("web_search", web_search_node)  # 웹 검색
workflow.add_node("retrieve", retrieve_node)  # 문서 검색
workflow.add_node("grade_documents", filtering_documents_node)  # 문서 평가
workflow.add_node("general_answer", general_answer_node)  # 일반 답변 생성
workflow.add_node("rag_answer", rag_answer_node)  # RAG 답변 생성

# 엣지 추가
workflow.add_conditional_edges(
    START,
    route_question_node,
    {
        "query_expansion": "query_expand",  # 웹 검색으로 라우팅
        "general_answer": "general_answer",  # 벡터스토어로 라우팅
    },
)

workflow.add_edge("query_expand", "retrieve")
workflow.add_edge("retrieve", "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_web_search_node,
    {
        "web_search": "web_search",  # 웹 검색 필요
        "rag_answer": "rag_answer",  # RAG 답변 생성 가능
    },
)

workflow.add_edge("query_rewrite", "rag_answer")

workflow.add_conditional_edges(
    "rag_answer",
    answer_groundedness_check,
    {
        "relevant": END,
        "not relevant": "web_search",
        "not grounded": "query_rewrite",
    },
)

workflow.add_edge("web_search", "rag_answer")


# 그래프 컴파일
app = workflow.compile(checkpointer=MemorySaver())

그래프를 시각화 합니다.

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(app)

## 그래프 실행

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = {
    "question": "Self-RAG 에서 사용되는 관련성 평가 노드 예제를 찾아줘",
}

# 스트리밍 형식으로 그래프 실행
stream_graph(
    app,
    inputs,
    config,
)